In [1]:
import kaolin as kal
import torch
import trimesh



In [2]:
def load_point_cloud(file_name):
    """ Load a mesh and sample points on its surface. """
    mesh = trimesh.load(file_name)  # Sample 10,000 points on the surface
    return mesh.vertices

In [3]:
# Load point clouds
pt_gt = torch.tensor(load_point_cloud('/home/rozenberszki/scene0423_02/scene0423_02_vh_clean_2.ply'))
#pt_comp = torch.tensor(load_point_cloud("/home/rozenberszki/project/wsnsl/output/Own/office4/mesh/final_mesh_color.ply")).to('cuda')
pt_comp = torch.tensor(load_point_cloud('/home/rozenberszki/project/wsnsl/output/scannet/scans/scene0423_02_panoptic/mesh/final_mesh_color.ply'))

In [4]:
import open3d as o3d
import numpy as np

# Load or create point clouds
# Convert the tensor to a numpy array
pt_gt_np = pt_gt.cpu().numpy()

# Create an Open3D point cloud
pt_gt_o3d = o3d.geometry.PointCloud()
pt_gt_o3d.points = o3d.utility.Vector3dVector(pt_gt_np)

pt_comp_np = pt_gt.cpu().numpy()

# Create an Open3D point cloud
pt_comp_o3d = o3d.geometry.PointCloud()
pt_comp_o3d.points = o3d.utility.Vector3dVector(pt_comp_np)



Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [5]:

# Apply ICP
threshold = 0.0002  # Set a threshold for point correspondence
trans_init = np.eye(4)  # Assume an initial guess of identity matrix
reg_p2p = o3d.pipelines.registration.registration_icp(
    pt_gt_o3d, pt_comp_o3d, threshold, trans_init,
    o3d.pipelines.registration.TransformationEstimationPointToPoint())

# Transform the second point cloud for alignment
pt_comp_o3d_aligned = pt_comp_o3d.transform(reg_p2p.transformation)

# Now pcd1 and pcd2 are aligned


In [6]:
import torch

# Convert the Open3D point cloud to a numpy array
pt_comp_aligned = np.asarray(pt_comp_o3d.points)

# Convert the numpy array to a tensor
pt_comp_aligned = torch.tensor(pt_comp_aligned)


In [7]:
pt_gt.min(dim=0)[0], pt_comp.min(dim=0)[0], pt_comp_aligned.min(dim=0)[0]

(tensor([1.3627, 1.4352, 0.0162], dtype=torch.float64),
 tensor([ 1.2500,  1.3500, -0.0400], dtype=torch.float64),
 tensor([1.3627, 1.4352, 0.0162], dtype=torch.float64))

In [12]:
x = pt_gt.unsqueeze(0).to('cuda')
y = pt_comp.unsqueeze(0).to('cuda')

In [13]:

# p1 and p2 are your point clouds
d1 = kal.metrics.pointcloud.chamfer_distance(x, y)
d1

tensor([0.1104], device='cuda:0', dtype=torch.float64)

In [14]:
d2 = kal.metrics.pointcloud.chamfer_distance(y, x)
d2

tensor([0.1104], device='cuda:0', dtype=torch.float64)

In [15]:
chamfer_dist = torch.mean(d1) + torch.mean(d2)
chamfer_dist

tensor(0.2207, device='cuda:0', dtype=torch.float64)

In [14]:
length = 100
line = "-" * length
print(line)


----------------------------------------------------------------------------------------------------


In [9]:
basepath_gt  = '/home/rozenberszki/project/replica_v2/{scene}/habitat/mesh_semantic.ply'
basepath_gt.format(scene='office_4')



'/home/rozenberszki/project/replica_v2/office_4/habitat/mesh_semantic.ply'

In [12]:
x = torch.tensor(x, dtype=torch.float32).to('cuda')
y = torch.tensor(y, dtype=torch.float32).to('cuda')
x = x.unsqueeze(0)
y = y.unsqueeze(0)
x.shape, y.shape

(torch.Size([1, 10000, 3]), torch.Size([1, 10000, 3]))

In [13]:

x_size = x.size(0)
y_size = y.size(0)
x_repeat = x.repeat(y_size, 1, 1)
y_repeat = y.repeat(1, x_size, 1)
x_repeat.shape, y_repeat.shape

(torch.Size([1, 10000, 3]), torch.Size([1, 10000, 3]))

In [ ]:
differences = torch.pow(x_repeat - y_repeat, 2).sum(2)
dist_x_to_y = torch.min(differences, 2)[0]
dist_y_to_x = torch.min(differences, 1)[0]

chamfer_dist = torch.mean(dist_x_to_y) + torch.mean(dist_y_to_x)

In [10]:

# Calculate Chamfer Distance
dist = chamfer_distance(points1, points2)
print(f"Chamfer Distance: {dist}")


IndexError: Dimension out of range (expected to be in range of [-2, 1], but got 2)